In [1]:
import sys
sys.path.append("/home/marin/projects/mhac/build/release")
import mhac

In [2]:
%matplotlib inline
import os, random, copy, math, csv, time
import matplotlib.pyplot as plt
import numpy as np

In [3]:
mhac.set_log_level(mhac.LogLevel.info)

In [4]:
def read_file_input(filepath):
    with open(filepath) as f:
        lines = f.readlines()
        sizes = [int(nr) for nr in lines[0].split()]
        m, n = sizes[0], sizes[1]
        arr = np.zeros((n, m))

        for i in range(1, len(lines)):
            arr[i-1, :] = [int(nr) for nr in lines[i].split()]

    return arr

In [5]:
# TODO: pybind should support numpy, so there might be a better way to transmit the data
def nparray_to_timematrix(arr):
    tm = mhac.problems.jss.TimeMatrix()

    for time_list in arr.tolist():
        vint = mhac.VectorInt()
        for duration in time_list:
            vint.append(int(duration))
        tm.append(vint)

    return tm

In [6]:
def read_all_files_in_folder(folder_path):
    # List to hold file contents
    file_contents = {}

    # Check if the provided path is a directory
    if not os.path.isdir(folder_path):
        raise ValueError(f"The provided path '{folder_path}' is not a valid directory.")

    # Iterate over all files in the directory
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)

        # Check if it's a file
        if os.path.isfile(file_path):
            try:
                # Open and read the file
                with open(file_path, 'r') as file:
                    content = file.read()
                    file_contents[filename] = content
            except Exception as e:
                print(f"Error reading file {filename}: {e}")
    
    return file_contents

In [7]:
def process_files(file_data, folder_path):
    results = []
    total_time = 0
    for filename, content in file_data.items():
        # problem = mhac.problems.jss.JSSP(nparray_to_timematrix(read_file_input(f"{folder_path}/{filename}")))
        # SA = mhac.physics.SimulatedAnnealing(problem)
        
        # problem = mhac.problems.jss.GA_JSSP(nparray_to_timematrix(read_file_input(f"{folder_path}/{filename}")))
        # GA = mhac.evolutionary.GeneticAlgorithm(problem)

        problem = mhac.problems.jss.ACO_JSSP(nparray_to_timematrix(read_file_input(f"{folder_path}/{filename}")))
        ACO = mhac.swarm.AntColonyOptimization(problem)

        start_time = time.time()

        # sol = SA.solve(maxT=1000, minT=0.000001, k=0.9995)
        # sol = GA.solve(generations=500, populationSize=20, mutationChance=0.75, selectionSize=5, selectionType=mhac.evolutionary.SelectionType.TOURNAMENT)
        sol = ACO.solve(generations=100, colonySize=20, alpha=0.55, beta=1.0, rho=0.15)

        end_time = time.time()
        duration = end_time - start_time

        print(f"Processing file: {filename} took {duration:.4f}s")
        print(f"Schedule: {sol.schedule} Cost: {sol.cost}")

        results.append({"filename": filename, "cost": sol.cost, "time": duration})
        total_time += duration

    print(f"Total time spent solving: {total_time:.4f}s")
    return results

In [8]:
def write_results_to_csv(results, output_file):
    with open(output_file, 'w', newline='') as csvfile:
        fieldnames = ['filename', 'cost', 'time']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()
        for result in results:
            writer.writerow(result)

In [9]:
folder_path = "../data/jss/imrg/ds1/testbed_1_s"
best_path = "../data/jss/imrg/ds1/best/testbed1_small.csv"
output_file = "results/mhac.csv"

In [10]:
file_data = read_all_files_in_folder(folder_path)

In [19]:
results = process_files(file_data, folder_path)

Processing file: t1s_0238.txt took 0.1218s
Schedule: VectorInt[4, 9, 3, 2, 1, 6, 8, 7, 0, 5] Cost: 11797.0
Processing file: t1s_0216.txt took 0.1153s
Schedule: VectorInt[1, 4, 8, 6, 2, 9, 0, 5, 7, 3] Cost: 12238.0
Processing file: t1s_0312.txt took 0.0839s
Schedule: VectorInt[7, 5, 1, 2, 10, 9, 6, 4, 8, 0, 3, 11] Cost: 11011.0
Processing file: t1s_0281.txt took 0.0590s
Schedule: VectorInt[10, 1, 0, 8, 4, 2, 3, 6, 11, 9, 5, 7] Cost: 6727.0
Processing file: t1s_0113.txt took 0.0722s
Schedule: VectorInt[2, 0, 3, 4, 1] Cost: 4745.0
Processing file: t1s_0082.txt took 0.0463s
Schedule: VectorInt[3, 0, 1, 4, 2] Cost: 2624.0
Processing file: t1s_0192.txt took 0.0782s
Schedule: VectorInt[0, 1, 3, 5, 6, 7, 8, 4, 2, 9] Cost: 8158.0
Processing file: t1s_0266.txt took 0.0388s
Schedule: VectorInt[2, 0, 8, 1, 6, 4, 10, 5, 11, 7, 3, 9] Cost: 4644.0
Processing file: t1s_0021.txt took 0.0231s
Schedule: VectorInt[0, 1, 4, 2, 3] Cost: 886.0
Processing file: t1s_0018.txt took 0.0208s
Schedule: VectorInt[2,

In [108]:
results.sort(key=lambda x: x["filename"])
write_results_to_csv(results, output_file)

# Results pure C++

## SA

| T | T_min | k | Time |
|---|-------|---|------|
| 1000 | 0.000001 | 0.9995 | 318s |
| 1000 | 0.001 | 0.9995 | 239s |
| 100 | 0.1 | 0.9995 | 139s |
| 100 | 0.1 | 0.999 | 53s |
| 100 | 0.1 | 0.995 | 11s |

## GA

| generations | time |
|-------------|------|
| 13800 | 3755s |
| 6900 | 1900s |
| 1400 | 445s |
| 1000 | 229s |
| 500 | 152s |
| 200 | 57s |
| 100 | 32s |

## ACO

| generations | time |
|-------------|------|
| 13800 | 3939s |
| 6900 | 2055s |
| 1400 | 424s |
| 1000 | 320s |
| 500 | 163s |
| 200 | 76s |
| 100 | 37s |

# Python problems and mhac

In [11]:
from queue import Queue

class PythonJSSP(mhac.common.Problem):
    def __init__(self, processing_times):
        super().__init__()
        self.processing_times = processing_times
        self.N = len(processing_times)
        self.M = len(processing_times[0])

    def generateInitialSolution(self):
        sol = mhac.problems.jss.JSSS()
        sol.schedule = random.sample(range(self.N), self.N)
        return sol

    def generateNewSolution(self, initialSol: mhac.problems.jss.JSSS):
        i, j = sorted(random.sample(range(self.N), 2))
        newSol = mhac.problems.jss.JSSS()
        newSol.schedule = initialSol.schedule  # copying with pybind, not using python refs
        # old_segment = newSol.tour[i:j+1]
        # newSol.tour[i:j+1] = old_segment[::-1]  # Slice and reverse the segment
        newSol.schedule[i], newSol.schedule[j] = newSol.schedule[j], newSol.schedule[i]
        return newSol

    def evaluateSolution(self, sol: mhac.problems.jss.JSSS):        
        # Initialize completion times matrix
        completion_times = [[0] * self.M for _ in range(self.N)]
        total_completion_time = 0

        for job_index in range(min(self.N, len(sol.schedule))):
            job = sol.schedule[job_index]

            for machine in range(self.M):
                if job_index == 0:
                    if machine == 0:
                        completion_times[job_index][machine] = self.processing_times[job][machine]
                    else:
                        completion_times[job_index][machine] = completion_times[job_index][machine-1] + self.processing_times[job][machine]
                else:
                    if machine == 0:
                        completion_times[job_index][machine] = completion_times[job_index-1][machine] + self.processing_times[job][machine]
                    else:
                        completion_times[job_index][machine] = self.processing_times[job][machine] + max(completion_times[job_index][machine-1], completion_times[job_index-1][machine])

                if machine == self.M - 1:
                    total_completion_time += completion_times[job_index][machine]

        return total_completion_time
    

class PythonJSSP_GA(mhac.evolutionary.Problem, PythonJSSP):
    def __init__(self, processing_times):
        mhac.evolutionary.Problem.__init__(self)
        PythonJSSP.__init__(self, processing_times)

    def repair(self, jss: mhac.problems.jss.JSSS):
        size = len(jss.schedule)

        seen = set()
        duplicates = []
        isPresent = [False] * size

        # Identify duplicates and check which elements are present
        for i in range(size):
            if jss.schedule[i] in seen:
                # This machine is a duplicate
                duplicates.append(i)
            else:
                seen.add(jss.schedule[i])
                isPresent[jss.schedule[i]] = True

        # Find missing elements
        missing = Queue()
        for i in range(size):
            if not isPresent[i]:
                missing.put(i)

        # Replace duplicates with missing elements
        for idx in duplicates:
            if not missing.empty():
                jss.schedule[idx] = missing.get()

        return jss
    
    def crossover(self, parent1, parent2):
        outChild1 = mhac.problems.jss.JSSS()
        outChild2 = mhac.problems.jss.JSSS()

        # Assuming schedule is a list within the JSSS objects
        cutPoint = random.randint(0, len(parent1.schedule) - 1)

        # Create children's schedules by slicing and appending parent schedules
        outChild1.schedule.extend(parent1.schedule[:cutPoint])
        outChild1.schedule.extend(parent2.schedule[cutPoint:])

        outChild2.schedule.extend(parent2.schedule[:cutPoint])
        outChild2.schedule.extend(parent1.schedule[cutPoint:])

        # Repair schedules
        outChild1 = self.repair(outChild1)
        outChild2 = self.repair(outChild2)
        
        # Evaluate costs
        outChild1.cost = self.evaluateSolution(outChild1)
        outChild2.cost = self.evaluateSolution(outChild2)

        res = mhac.common.SolutionVec()
        res.append(outChild1)
        res.append(outChild2)

        return res

    def mutation(self, outChild, mutationChance):
        i, j = random.sample(range(len(outChild.schedule)), 2)

        if random.random() < mutationChance:
            outChild.schedule[i], outChild.schedule[j] = outChild.schedule[j], outChild.schedule[i]
            outChild.cost = self.evaluateSolution(outChild)
        
        return outChild


class PythonJSSP_ACO(mhac.swarm.Problem, PythonJSSP):
    def __init__(self, processing_times):
        mhac.swarm.Problem.__init__(self)
        PythonJSSP.__init__(self, processing_times)

    def updateAntPath(self, ant, pm, alpha, beta):
        schedule_size = len(ant.schedule)

        # print("start schedule", ant.schedule)
        
        for i in range(schedule_size):
            probabilities = np.zeros(schedule_size)
            sum_probabilities = 0.0
            availableJobs = ant.schedule[i:]
            # print("availableJobs", availableJobs)

            for j in availableJobs:
                pheromone = pm(ant.schedule[i], j)**alpha

                ant_prime = mhac.problems.jss.JSSS()
                ant_prime.schedule = ant.schedule[:i+1]
                ant_prime.schedule.append(j)
                # print("ant_prime", ant_prime.schedule)

                ant_prime.cost = self.evaluateSolution(ant_prime)

                heuristic = 1.0 / ant_prime.cost
                
                eta = heuristic ** beta
                probabilities[j] = pheromone * eta

                sum_probabilities += probabilities[j]

                for j in availableJobs:
                    probabilities[j] /= sum_probabilities

            selected_index = random.choices(range(len(probabilities)), weights=probabilities)[0]
            ant.schedule[i] = selected_index
        
        return ant

    def updatePheromoneMatrix(self, ant, pm, rho):
        total_job_completion_time = self.evaluateSolution(ant)
        deposit = 1.0 / total_job_completion_time

        # Evaporate the existing pheromone
        size = pm.getSize()
        for i in range(size):
            for j in range(size):
                current_value = pm(i, j)
                pm.set(i, j, current_value * (1 - rho))

        # Deposit new pheromones based on the ant's path
        for k in range(len(ant.schedule) - 1):
            i = ant.schedule[k]
            j = ant.schedule[k + 1]
            pm.set(i, j, pm(i, j) + deposit)

In [12]:
def process_files_pyjssp(file_data):
    results = []
    total_time = 0
    for filename, content in file_data.items():
        # problem = PythonJSSP(nparray_to_timematrix(read_file_input(f"{folder_path}/{filename}")))
        # SA = mhac.physics.SimulatedAnnealing(problem)

        # problem = PythonJSSP_GA(nparray_to_timematrix(read_file_input(f"{folder_path}/{filename}")))
        # GA = mhac.evolutionary.GeneticAlgorithm(problem)

        problem = PythonJSSP_ACO(nparray_to_timematrix(read_file_input(f"{folder_path}/{filename}")))
        ACO = mhac.swarm.AntColonyOptimization(problem)

        start_time = time.time()
        print(f"Processing file: {filename}")

        # sol = SA.solve(maxT=100, minT=0.1, k=0.995)
        # sol = GA.solve(generations=1400, populationSize=20, mutationChance=0.75, selectionSize=5, selectionType=mhac.evolutionary.SelectionType.TOURNAMENT)
        sol = ACO.solve(generations=100, colonySize=4, alpha=0.55, beta=1.0, rho=0.15)
        
        end_time = time.time()
        duration = end_time - start_time

        print(f"Processing file: {filename} took {duration:.4f}s")
        print(f"Schedule: {sol.schedule} Cost: {sol.cost}")

        results.append({"filename": filename, "cost": sol.cost, "time": duration})
        total_time += duration

    print(f"Total time spent solving: {total_time:.4f}s")
    return results

In [13]:
results = process_files_pyjssp(file_data)

Processing file: t1s_0238.txt
Processing file: t1s_0238.txt took 6.4176s
Schedule: VectorInt[9, 2, 8, 6, 0, 3, 7, 4, 1, 5] Cost: 13173.0
Processing file: t1s_0216.txt
Processing file: t1s_0216.txt took 5.8127s
Schedule: VectorInt[4, 8, 7, 9, 2, 0, 5, 3, 6, 1] Cost: 13637.0
Processing file: t1s_0312.txt
Processing file: t1s_0312.txt took 5.1177s
Schedule: VectorInt[4, 10, 9, 3, 1, 8, 5, 11, 2, 7, 6, 0] Cost: 11261.0
Processing file: t1s_0281.txt
Processing file: t1s_0281.txt took 3.2974s
Schedule: VectorInt[10, 8, 1, 0, 6, 4, 3, 9, 5, 2, 11, 7] Cost: 6995.0
Processing file: t1s_0113.txt
Processing file: t1s_0113.txt took 1.3387s
Schedule: VectorInt[0, 3, 4, 2, 1] Cost: 6157.0
Processing file: t1s_0082.txt
Processing file: t1s_0082.txt took 0.7178s
Schedule: VectorInt[1, 0, 4, 2, 3] Cost: 3271.0
Processing file: t1s_0192.txt
Processing file: t1s_0192.txt took 3.3260s
Schedule: VectorInt[1, 3, 4, 2, 6, 9, 0, 5, 8, 7] Cost: 8611.0
Processing file: t1s_0266.txt
Processing file: t1s_0266.txt

# Results C++ and python operators

## SA

| T | T_min | k | Time |
|---|-------|---|------|
| 1000 | 0.000001 | 0.9995 | 2725s |
| 1000 | 0.001 | 0.9995 | 1751s |
| 100 | 0.1 | 0.9995 | 876s |
| 100 | 0.1 | 0.999 | 500s |
| 100 | 0.1 | 0.995 | 90s |

## GA

| generations | time |
|-------------|------|
| 1400 | 3796s |
| 1000 | 2637s |
| 500 | 1369s |
| 200 | 628s |
| 100 | 311s |


## ACO

| generations | time |
|-------------|------|
| 1400 | 6421s |
| 1000 | 4257s |
| 500 | 2091s |
| 200 | 915s |
| 100 | 340s |

# Tuning Params

In [15]:
from skopt.space import Real, Integer
from skopt import gp_minimize

In [30]:
def compute_overall_score_SA(params):
    # maxT, minT, k = params
    k = params[0]
    total_score = 0

    for filename, content in file_data.items():
        problem = mhac.problems.jss.JSSP(nparray_to_timematrix(read_file_input(f"{folder_path}/{filename}")))
        SA = mhac.physics.SimulatedAnnealing(problem)
        sol = SA.solve(maxT=100, minT=0.1, k=k)
        total_score += sol.cost

    return total_score

In [31]:
# Define the search space for the parameters
space = [
    # Integer(50, 150, name='maxT'),
    # Real(0.0001, 0.01, name='minT'),
    Real(0.95, 0.9995, name='k')
]

# Perform Bayesian optimization
res = gp_minimize(compute_overall_score_SA, space, n_calls=100, random_state=0)

/home/marin/.local/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [0.9995] before, using random point [0.9909597860700232]
  warnings.warn(
/home/marin/.local/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [0.9995] before, using random point [0.9639316539967145]
  warnings.warn(
/home/marin/.local/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [0.9995] before, using random point [0.9662968120004071]
  warnings.warn(
/home/marin/.local/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [0.9995] before, using random point [0.956299163965881]
  warnings.warn(
/home/marin/.local/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [0.9995] before, using ra

In [33]:
# Output the best parameters found
print("Best parameters found:")
# print(f"maxT: {res.x[0]}")
# print(f"minT: {res.x[1]}")
# print(f"k: {res.x[2]}")
print(f"k: {res.x[0]}")

print("Best score achieved:")
print(res.fun)

Best parameters found:
k: 0.9995
Best score achieved:
2107919.0


In [36]:
def compute_overall_score_GA(params):
    # mutationChance, generations, populationSize, selectionSize = params
    mutationChance, selectionSize = params
    total_score = 0

    for filename, content in file_data.items():
        problem = PythonJSSP_GA(nparray_to_timematrix(read_file_input(f"{folder_path}/{filename}")))
        GA = mhac.evolutionary.GeneticAlgorithm(problem)
        sol = GA.solve(generations=100, populationSize=20, mutationChance=mutationChance, selectionSize=selectionSize, selectionType=mhac.evolutionary.SelectionType.TOURNAMENT)
        total_score += sol.cost

    return total_score

In [37]:
# Define the search space for the parameters
space = [
    Real(0.1, 1.0, name='mutationChance'),
    # Integer(25, 100, name='generations'),
    # Integer(5,   20, name='populationSize'),
    Integer(5,   20, name='selectionSize')
]

# Perform Bayesian optimization
res = gp_minimize(compute_overall_score_GA, space, n_calls=100, random_state=0)

In [38]:
# Output the best parameters found
print("Best parameters found:")
# print(f"mutationChance: {res.x[0]}")
# print(f"generations: {res.x[1]}")
# print(f"populationSize: {res.x[2]}")
# print(f"selectionSize: {res.x[3]}")
print(f"mutationChance: {res.x[0]}")
print(f"selectionSize: {res.x[1]}")

print("Best score achieved:")
print(res.fun)

Best parameters found:
mutationChance: 0.7537086574805754
selectionSize: 6
Best score achieved:
2116635.0


In [39]:
def compute_overall_score_ACO(params):
    # alpha, beta, rho, generations, colonySize = params
    alpha, beta, rho = params
    total_score = 0

    for filename, content in file_data.items():
        problem = PythonJSSP_ACO(nparray_to_timematrix(read_file_input(f"{folder_path}/{filename}")))
        ACO = mhac.swarm.AntColonyOptimization(problem)
        sol = ACO.solve(generations=100, colonySize=20, alpha=alpha, beta=beta, rho=rho)
        total_score += sol.cost

    return total_score

In [40]:
# Define the search space for the parameters
space = [
    Real(0.5, 2.0, name='alpha'),
    Real(1.0, 5.0, name='beta'),
    Real(0.1, 0.5, name='rho'),
    # Integer(25, 100, name='generations'),
    # Integer(5, 20, name='colonySize')
]

# Perform Bayesian optimization
res = gp_minimize(compute_overall_score_ACO, space, n_calls=100, random_state=0)

/home/marin/.local/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [0.5, 1.0, 0.1] before, using random point [1.5560341295049893, 1.0351991319826248, 0.2720392596834276]
  warnings.warn(
/home/marin/.local/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [0.5, 1.0, 0.1] before, using random point [1.5684694239682795, 4.660682727774938, 0.18111954579918876]
  warnings.warn(
/home/marin/.local/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [0.5, 1.0, 0.1] before, using random point [0.9463660775330068, 3.5620684542648817, 0.14089948123309254]
  warnings.warn(
/home/marin/.local/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [0.5, 1.0, 0.1] before, using random point [1.2577619603720287, 1.3559938401193041, 0.337343

In [41]:
# Output the best parameters found
print("Best parameters found:")
print(f"alpha: {res.x[0]}")
print(f"beta: {res.x[1]}")
print(f"rho: {res.x[2]}")
# print(f"generations: {res.x[3]}")
# print(f"colony_size: {res.x[4]}")

print("Best score achieved:")
print(res.fun)

Best parameters found:
alpha: 0.5221307439787357
beta: 1.0044132437154545
rho: 0.13759002280210966
Best score achieved:
2220121.0
